# 2. Machine Learning Pipeline - Home Credit Default Risk: Feature Engineering

### Imports 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils import agg_stats_data, split_with_stratified_shuffle_split

## 2. Feature Engineering

In [2]:
ROOTH_PATH = os.getcwd()
RAW_DATA_PATH = os.path.join(ROOTH_PATH, 'data/raw')
PROCESSED_DATA_PATH = os.path.join(ROOTH_PATH, "data/processed")

TARGET = 'TARGET'
RANDOM_STATE = 123456
TEST_SIZE = 0.20
N_SPLITS = 2

In [7]:
train_set = pd.read_csv(os.path.join(RAW_DATA_PATH, 'application_train.csv'))
train_set['AGE']  = train_set['DAYS_BIRTH'] / - 365
train_set.drop(columns='DAYS_BIRTH')
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 123 entries, SK_ID_CURR to AGE
dtypes: float64(66), int64(41), object(16)
memory usage: 288.6+ MB


### Bureau

In [8]:
bureau = pd.read_csv(os.path.join(RAW_DATA_PATH, "bureau.csv"))
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [9]:
bureau_agg = agg_stats_data(bureau, 'SK_ID_CURR')
bureau_agg.head()

Number categorical variables: 3
Number numerical variables: 13


,SK_ID_CURR,SK_ID_BUREAU_count,SK_ID_BUREAU_mean,SK_ID_BUREAU_max,SK_ID_BUREAU_min,SK_ID_BUREAU_sum,DAYS_CREDIT_count,DAYS_CREDIT_mean,DAYS_CREDIT_max,DAYS_CREDIT_min,...,CREDIT_TYPE_Interbank credit_count,CREDIT_TYPE_Loan for business development_count,CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,CREDIT_TYPE_Loan for the purchase of equipment_count,CREDIT_TYPE_Loan for working capital replenishment_count,CREDIT_TYPE_Microloan_count,CREDIT_TYPE_Mobile operator loan_count,CREDIT_TYPE_Mortgage_count,CREDIT_TYPE_Real estate loan_count,CREDIT_TYPE_Unknown type of loan_count
0,100001,7,5896633.000,5896636,5896630,41276431,7,-735.000000,-49,-1572,...,7,7,7,7,7,7,7,7,7,7
1,100002,8,6153272.125,6158909,6113835,49226177,8,-874.000000,-103,-1437,...,8,8,8,8,8,8,8,8,8,8
2,100003,4,5885878.500,5885880,5885877,23543514,4,-1400.750000,-606,-2586,...,4,4,4,4,4,4,4,4,4,4
3,100004,2,6829133.500,6829134,6829133,13658267,2,-867.000000,-408,-1326,...,2,2,2,2,2,2,2,2,2,2
4,100005,3,6735201.000,6735202,6735200,20205603,3,-190.666667,-62,-373,...,3,3,3,3,3,3,3,3,3,3


### Bureau balance

In [10]:
bureau_balance = pd.read_csv(os.path.join(RAW_DATA_PATH, 'bureau_balance.csv'))
bureau_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [11]:
bureau_balance_agg = agg_stats_data(bureau_balance, 'SK_ID_BUREAU')
bureau_balance_agg.head()

Number categorical variables: 1
Number numerical variables: 1


,SK_ID_BUREAU,MONTHS_BALANCE_count,MONTHS_BALANCE_mean,MONTHS_BALANCE_max,MONTHS_BALANCE_min,MONTHS_BALANCE_sum,STATUS_0_count,STATUS_1_count,STATUS_2_count,STATUS_3_count,STATUS_4_count,STATUS_5_count,STATUS_C_count,STATUS_X_count
0,5001709,97,-48.0,0,-96,-4656,97,97,97,97,97,97,97,97
1,5001710,83,-41.0,0,-82,-3403,83,83,83,83,83,83,83,83
2,5001711,4,-1.5,0,-3,-6,4,4,4,4,4,4,4,4
3,5001712,19,-9.0,0,-18,-171,19,19,19,19,19,19,19,19
4,5001713,22,-10.5,0,-21,-231,22,22,22,22,22,22,22,22


### Previous Application

In [12]:
previous_application = pd.read_csv(os.path.join(RAW_DATA_PATH, 'previous_application.csv'))
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
previous_application_agg = agg_stats_data(previous_application, 'SK_ID_CURR')
previous_application_agg.head()

Number categorical variables: 16
Number numerical variables: 20


,SK_ID_CURR,SK_ID_PREV_count,SK_ID_PREV_mean,SK_ID_PREV_max,SK_ID_PREV_min,SK_ID_PREV_sum,AMT_ANNUITY_count,AMT_ANNUITY_mean,AMT_ANNUITY_max,AMT_ANNUITY_min,...,PRODUCT_COMBINATION_Cash X-Sell: low_count,PRODUCT_COMBINATION_Cash X-Sell: middle_count,PRODUCT_COMBINATION_POS household with interest_count,PRODUCT_COMBINATION_POS household without interest_count,PRODUCT_COMBINATION_POS industry with interest_count,PRODUCT_COMBINATION_POS industry without interest_count,PRODUCT_COMBINATION_POS mobile with interest_count,PRODUCT_COMBINATION_POS mobile without interest_count,PRODUCT_COMBINATION_POS other with interest_count,PRODUCT_COMBINATION_POS others without interest_count
0,100001,1,1.369693e+06,1369693,1369693,1369693,1,3951.000,3951.000,3951.000,...,1,1,1,1,1,1,1,1,1,1
1,100002,1,1.038818e+06,1038818,1038818,1038818,1,9251.775,9251.775,9251.775,...,1,1,1,1,1,1,1,1,1,1
2,100003,3,2.281150e+06,2636178,1810518,6843451,3,56553.990,98356.995,6737.310,...,3,3,3,3,3,3,3,3,3,3
3,100004,1,1.564014e+06,1564014,1564014,1564014,1,5357.250,5357.250,5357.250,...,1,1,1,1,1,1,1,1,1,1
4,100005,2,2.176837e+06,2495675,1857999,4353674,1,4813.200,4813.200,4813.200,...,2,2,2,2,2,2,2,2,2,2


### Monthly Cash Data

In [14]:
cash = pd.read_csv(os.path.join(RAW_DATA_PATH, 'POS_CASH_balance.csv'))
cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [15]:
cash_agg = agg_stats_data(cash, 'SK_ID_CURR')
cash_agg.head()

Number categorical variables: 1
Number numerical variables: 6


,SK_ID_CURR,SK_ID_PREV_count,SK_ID_PREV_mean,SK_ID_PREV_max,SK_ID_PREV_min,SK_ID_PREV_sum,MONTHS_BALANCE_count,MONTHS_BALANCE_mean,MONTHS_BALANCE_max,MONTHS_BALANCE_min,...,SK_DPD_DEF_sum,NAME_CONTRACT_STATUS_Active_count,NAME_CONTRACT_STATUS_Amortized debt_count,NAME_CONTRACT_STATUS_Approved_count,NAME_CONTRACT_STATUS_Canceled_count,NAME_CONTRACT_STATUS_Completed_count,NAME_CONTRACT_STATUS_Demand_count,NAME_CONTRACT_STATUS_Returned to the store_count,NAME_CONTRACT_STATUS_Signed_count,NAME_CONTRACT_STATUS_XNA_count
0,100001,9,1.584045e+06,1851984,1369693,14256401,9,-72.555556,-53,-96,...,7,9,9,9,9,9,9,9,9,9
1,100002,19,1.038818e+06,1038818,1038818,19737542,19,-10.000000,-1,-19,...,0,19,19,19,19,19,19,19,19,19
2,100003,28,2.297665e+06,2636178,1810518,64334628,28,-43.785714,-18,-77,...,0,28,28,28,28,28,28,28,28,28
3,100004,4,1.564014e+06,1564014,1564014,6256056,4,-25.500000,-24,-27,...,0,4,4,4,4,4,4,4,4,4
4,100005,11,2.495675e+06,2495675,2495675,27452425,11,-20.000000,-15,-25,...,0,11,11,11,11,11,11,11,11,11


### Monthly Credit Data

In [16]:
credit = pd.read_csv(os.path.join(RAW_DATA_PATH, 'credit_card_balance.csv')) 
credit.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [17]:
credit_agg = agg_stats_data(credit, 'SK_ID_CURR')
credit_agg.head()

Number categorical variables: 1
Number numerical variables: 21


,SK_ID_CURR,SK_ID_PREV_count,SK_ID_PREV_mean,SK_ID_PREV_max,SK_ID_PREV_min,SK_ID_PREV_sum,MONTHS_BALANCE_count,MONTHS_BALANCE_mean,MONTHS_BALANCE_max,MONTHS_BALANCE_min,...,SK_DPD_DEF_max,SK_DPD_DEF_min,SK_DPD_DEF_sum,NAME_CONTRACT_STATUS_Active_count,NAME_CONTRACT_STATUS_Approved_count,NAME_CONTRACT_STATUS_Completed_count,NAME_CONTRACT_STATUS_Demand_count,NAME_CONTRACT_STATUS_Refused_count,NAME_CONTRACT_STATUS_Sent proposal_count,NAME_CONTRACT_STATUS_Signed_count
0,100006,6,1489396.0,1489396,1489396,8936376,6,-3.5,-1,-6,...,0,0,0,6,6,6,6,6,6,6
1,100011,74,1843384.0,1843384,1843384,136410416,74,-38.5,-2,-75,...,0,0,0,74,74,74,74,74,74,74
2,100013,96,2038692.0,2038692,2038692,195714432,96,-48.5,-1,-96,...,1,0,1,96,96,96,96,96,96,96
3,100021,17,2594025.0,2594025,2594025,44098425,17,-10.0,-2,-18,...,0,0,0,17,17,17,17,17,17,17
4,100023,8,1499902.0,1499902,1499902,11999216,8,-7.5,-4,-11,...,0,0,0,8,8,8,8,8,8,8


### Installment Payments

In [18]:
installments = pd.read_csv(os.path.join(RAW_DATA_PATH, 'installments_payments.csv'))
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [19]:
installments_agg = agg_stats_data(installments, 'SK_ID_CURR')
installments_agg.head()

Number categorical variables: 0
Number numerical variables: 7


,SK_ID_CURR,SK_ID_PREV_count,SK_ID_PREV_mean,SK_ID_PREV_max,SK_ID_PREV_min,SK_ID_PREV_sum,NUM_INSTALMENT_VERSION_count,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_VERSION_min,...,AMT_INSTALMENT_count,AMT_INSTALMENT_mean,AMT_INSTALMENT_max,AMT_INSTALMENT_min,AMT_INSTALMENT_sum,AMT_PAYMENT_count,AMT_PAYMENT_mean,AMT_PAYMENT_max,AMT_PAYMENT_min,AMT_PAYMENT_sum
0,100001,7,1.576389e+06,1851984,1369693,11034724,7,1.142857,2.0,1.0,...,7,5885.132143,17397.900,3951.000,41195.925,7,5885.132143,17397.900,3951.000,41195.925
1,100002,19,1.038818e+06,1038818,1038818,19737542,19,1.052632,2.0,1.0,...,19,11559.247105,53093.745,9251.775,219625.695,19,11559.247105,53093.745,9251.775,219625.695
2,100003,25,2.290070e+06,2636178,1810518,57251754,25,1.040000,2.0,1.0,...,25,64754.586000,560835.360,6662.970,1618864.650,25,64754.586000,560835.360,6662.970,1618864.650
3,100004,3,1.564014e+06,1564014,1564014,4692042,3,1.333333,2.0,1.0,...,3,7096.155000,10573.965,5357.250,21288.465,3,7096.155000,10573.965,5357.250,21288.465
4,100005,9,2.495675e+06,2495675,2495675,22461075,9,1.111111,2.0,1.0,...,9,6240.205000,17656.245,4813.200,56161.845,9,6240.205000,17656.245,4813.200,56161.845


In [20]:
df = pd.merge(train_set, bureau_agg, on='SK_ID_CURR', how='left')
#df = pd.merge(df, bureau_balance_agg, on='SK_ID_BUREAU', how='inner')
df = pd.merge(df, previous_application_agg, on='SK_ID_CURR', how='left')
df = pd.merge(df, cash_agg, on='SK_ID_CURR', how='left')
df = pd.merge(df, credit_agg, on='SK_ID_CURR', how='left')
df = pd.merge(df, installments_agg, on='SK_ID_CURR', how='left')
df.shape

/var/folders/gn/fvzh0_yx1y74zlkhtnq2_j_w0000gn/T/ipykernel_27664/3410184586.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'SK_ID_PREV_count_x', 'SK_ID_PREV_min_x', 'SK_ID_PREV_max_x', 'SK_ID_PREV_sum_x', 'SK_ID_PREV_mean_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, installments_agg, on='SK_ID_CURR', how='left')


(307511, 640)

In [21]:
df = df.drop(columns=[
    'NAME_TYPE_SUITE_Spouse, partner_count',
    'PRODUCT_COMBINATION_Cash Street: high_count',
    'PRODUCT_COMBINATION_Cash Street: low_count',
    'PRODUCT_COMBINATION_Cash Street: middle_count',
    'PRODUCT_COMBINATION_Cash X-Sell: high_count',
    'PRODUCT_COMBINATION_Cash X-Sell: low_count',
    'PRODUCT_COMBINATION_Cash X-Sell: middle_count',
])
df.shape

(307511, 633)

## 3. Split data

#### Split data on train_set and train_test

In [23]:
train_info, test_info = split_with_stratified_shuffle_split(df, TARGET, N_SPLITS, TEST_SIZE, RANDOM_STATE)
train_info.to_csv(os.path.join(PROCESSED_DATA_PATH, "train_info.csv"))
test_info.to_csv(os.path.join(PROCESSED_DATA_PATH, "test_info.csv"))

61503
61503
Data Train info: 246008
Data Test info: 61503
